# Technical Report

## Overview
This notebook walks through the methodology of our ETL process for the video games sales database. 

Both of the data sets are available for download at [kaggle](https://www.kaggle.com/). 

The first decision we made was to use a PostgreSQL database. Relational database are typically easier to query. They can be harder to set up and limit what type of data can be added to the database. Since the data sets we are working with already have some relational structure, using SQL as opposed to MongoDB or another non-relational database is a natural choioce. We created our database's ERD using [quickDBD](https://www.quickdatabasediagrams.com/). The database is in 1st normal form - every element is atomic and cannot be borken down further. This fascilitates easier data querying and manipulation. The database is not in 2nd normal form because the "sales" table lacks a single column primary key. Since this would not drastically impact the functionality, we decided against it to keep the model more simple. 

<spelling>

![erd](project-2_database_erd.png)

## Extract
The extraction step is straight forward; both data sets are downloaded as csv files. First we laod them as dataframes using pandas csv reader and them check their contents for non-null data and make sure they make sense. 

In [8]:
# import pandas
import pandas as pd

# define the paths to the csv files that contain the raw data
world_pop_path = "Resources/world_population.csv"
vg_sales_path = "Resources/vgsales.csv"

# read the population data as a dataframe  and preview it
world_pop_df = pd.read_csv(world_pop_path)
world_pop_df.head()

,Rank,CCA3,Country,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56
3,213,ASM,American Samoa,Pago Pago,Oceania,44273,46189,51368,54849,58230,47818,32886,27075,199,222.4774,0.9831,0.00
4,203,AND,Andorra,Andorra la Vella,Europe,79824,77700,71746,71519,66097,53569,35611,19860,468,170.5641,1.0100,0.00


In [9]:
# view data types and amount of non-null data per column
world_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Rank                         234 non-null    int64  
 1   CCA3                         234 non-null    object 
 2   Country                      234 non-null    object 
 3   Capital                      234 non-null    object 
 4   Continent                    234 non-null    object 
 5   2022 Population              234 non-null    int64  
 6   2020 Population              234 non-null    int64  
 7   2015 Population              234 non-null    int64  
 8   2010 Population              234 non-null    int64  
 9   2000 Population              234 non-null    int64  
 10  1990 Population              234 non-null    int64  
 11  1980 Population              234 non-null    int64  
 12  1970 Population              234 non-null    int64  
 13  Area (km²)          

The world population data contains 17 columns and all of them appear to contain 234 non-null values. In the transform step we will drop quite a bit of this data and clean it up so we are left with the data that is most useful for the project.

In [5]:
# read the video games data as a dataframe and preview it
vg_sales_df = pd.read_csv(vg_sales_path)
vg_sales_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [10]:
# view data types and amount of non-null data per column
vg_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


The video games sales data contains 11 columns with 16,598 non-null calues in all the columns except publisher and release year. We could drop rows with incomplete data. However, for this project we are only concerned about sales data. So we keep all data as is and will drop unused columns in the transform process. 

## Transform
First we work on creating the world population table. The table needs to store the population of each region - Japan, North America, Europe and the world. For this project, we only need population data for 2020. This is because, our sales data are cumulative up unitl 2020.

Japan's population is in the dataframe already so it can be loaded as is. To get the populations for each continent, we take the data and aggregate-sum it by continent. Finally, to get the world population, we sum over all the continents.

In [7]:
# group and sum all data by cntinent
world_pop_by_continents = world_pop_df.groupby(world_pop_df['Continent']).sum()
world_pop_by_continents

,Rank,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
Continent,,,,,,,,,,,,,
Africa,5253,1426730932,1360671810,1201102442,1055228072,818946032,638150629,481536377,365444348,30317963,7127.7158,58.2109,17.87
Asia,3878,4721383274,4663086535,4458250182,4220041327,3735089604,3210563577,2635334228,2144906290,32138141,51251.2068,50.4692,59.19
Europe,6225,743147538,745792196,741535608,735613934,726093423,720320797,692527159,655923991,23010411,33166.2371,50.1128,9.33
North America,6437,600296136,594236593,570383850,542720651,486069584,421266425,368293361,315434606,24244178,10910.4703,40.1670,7.51
Oceania,4336,45038554,43933426,40403283,37102764,31222778,26743822,22920240,19480270,8515081,3048.4905,23.1698,0.55
South America,1366,436816608,431530043,413134396,393078250,349634282,297146415,241789006,192947156,17833382,293.6077,14.1114,5.48


In [12]:
# extract values for population table

# na and eu populations are taken from the group by dataframe
na_pop = world_pop_by_continents.loc['North America', '2020 Population']
eu_pop = world_pop_by_continents.loc['Europe', '2020 Population']

# wo population is calculated by summing over all continents
wo_pop = world_pop_by_continents['2020 Population'].sum()

# jp is seelected from the original df
jp_pop = world_pop_df.loc[world_pop_df['Country']== 'Japan', '2020 Population'].values[0]

#save region populations into table
population = pd.DataFrame({"Region":["na", "eu", "jp", "wo"], "Population":[na_pop, eu_pop, jp_pop, wo_pop]})
population

,Region,Population
0,na,594236593
1,eu,745792196
2,jp,125244761
3,wo,7839250603


This is the final table, ready to be loaded into the database. It's primary key is region and the populations for each region are stored as whole numbers. 

Next we work on the video games table. Each row of the dataframe already represents a unique video game and platform pair. So we just need to assign a primary key to each row of the original dataframe and retain the video game and platform data. The column 'rank' from the original dataframe idicates the game, platform pair's rank in total sales. While we are not interested in this number

In [13]:
# rename the rank cloumn as 'Game_Id'
vg_sales_df= vg_sales_df.rename(columns={"Rank": "Game_Id"})

# define the video games df 
video_games = vg_sales_df[["Game_Id", "Name", "Platform"]]
video_games.head()

,Game_Id,Name,Platform
0,1,Wii Sports,Wii
1,2,Super Mario Bros.,NES
2,3,Mario Kart Wii,Wii
3,4,Wii Sports Resort,Wii
4,5,Pokemon Red/Pokemon Blue,GB


This is the video_games table ready to load into the database. 

Lastly we create the sales table. 

In [ ]:
# The melt function takes data from multpile columns and stores it as seperate elements
sales=vg_sales_df.melt(value_vars=["NA_Sales", "EU_Sales", "JP_Sales", "Global_Sales"],id_vars=["Game_Id"], 
                       var_name="Region", value_name="Sales")

# 
sales["Region"].replace({"NA_Sales": "na", "EU_Sales": "eu", "JP_Sales": "jp", "Global_Sales": "wo"}, inplace=True)

sales["Sales"]= sales["Sales"]*1000000

sales.head()

## Load

In [14]:
# import methods sqlalchemy
from sqlalchemy import create_engine, inspect

# 
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'project-2_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

NameError: name 'create_engine' is not defined